In [2]:
import pandas as pd 
import numpy as np
import datetime as dt

In [3]:
df = pd.read_csv("../data/df_final_metadata.csv")
df

,fecha_hora,cmg,demanda,gx_Eólicas_Antofagasta,gx_Eólicas_Araucanía,gx_Eólicas_Atacama,gx_Eólicas_Biobío,gx_Eólicas_Coquimbo,gx_Eólicas_Los Lagos,gx_Eólicas_O’Higgins,...,emb_MELADO,emb_PANGUE,emb_POLCURA,emb_RALCO,emb_RAPEL,year,month,day,day_of_week,hour
0,2019-01-01 00:00:00,55.52,7992.914520,89.92,71.65,26.40,57.09,459.92,26.54,3.03,...,643.47,509.02,735.55,715.75,103.52,2019,1,1,1,0
1,2019-01-01 01:00:00,55.52,7948.664819,72.62,79.50,10.70,58.19,388.12,22.47,1.76,...,643.63,508.90,735.57,715.75,103.52,2019,1,1,1,1
2,2019-01-01 02:00:00,55.52,7752.115956,35.16,77.97,1.90,48.44,304.78,13.95,0.65,...,643.79,508.81,735.60,715.73,103.52,2019,1,1,1,2
3,2019-01-01 03:00:00,55.52,7523.577425,17.75,66.52,0.00,39.39,252.67,13.50,1.51,...,643.94,508.74,735.62,715.73,103.52,2019,1,1,1,3
4,2019-01-01 04:00:00,55.52,7308.782546,8.89,51.07,0.00,41.75,278.55,15.77,3.25,...,644.08,508.75,735.62,715.73,103.52,2019,1,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48131,2024-06-28 15:00:00,0.00,9228.687500,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,645.91,508.77,735.03,721.98,104.52,2024,6,28,4,15
48132,2024-06-28 16:00:00,9.03,9903.786133,474.64,262.98,127.24,288.62,286.39,5.97,36.77,...,645.92,508.71,735.07,722.04,104.53,2024,6,28,4,16
48133,2024-06-28 17:00:00,35.36,10463.607422,494.72,477.75,74.94,317.90,178.76,6.89,69.48,...,645.83,508.72,734.94,722.07,104.50,2024,6,28,4,17
48134,2024-06-28 18:00:00,69.48,10986.050781,303.65,654.46,28.13,365.90,85.30,2.12,71.48,...,645.78,508.71,734.92,722.07,104.51,2024,6,28,4,18


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48136 entries, 0 to 48135
Columns: 112 entries, fecha_hora to date
dtypes: datetime64[ns](1), float64(105), int64(5), object(1)
memory usage: 41.1+ MB


In [31]:
# Assuming your DataFrame is named df and has a 'fecha_hora' and 'cmg' column
# Convert datetime column to datetime type if it's not already
df['fecha_hora'] = pd.to_datetime(df['fecha_hora'])

# Extract the date (without time) for each row
df['date'] = df['fecha_hora'].dt.date

# Initialize an empty list to store the augmented rows
augmented_rows = []

# Iterate over each unique date (at 00:00:00) to get 32-hour windows
for date in df['date'].unique():
    # Get the rows corresponding to this date
    start_datetime = dt.datetime.combine(date, dt.time(0, 0, 0))

    # Extract the 32-hour window
    window = df[(df['fecha_hora'] >= start_datetime) & 
                (df['fecha_hora'] < start_datetime + pd.Timedelta(hours=32))].copy()
    
    # Rank the 'cmg' column within the window
    window['rank'] = window['cmg'].rank(ascending=False,method='first')

    # Append the window with 'date' and 'rank' to the augmented rows
    augmented_rows.append(window)

# Concatenate all augmented rows into a new DataFrame
augmented_df = pd.concat(augmented_rows)
augmented_df['rank'] = augmented_df['rank'].astype(int)

# Reset the index
augmented_df.reset_index(drop=True, inplace=True)

In [32]:
augmented_df[augmented_df['date'] == pd.to_datetime('2022-12-01').date()]

,fecha_hora,cmg,demanda,gx_Eólicas_Antofagasta,gx_Eólicas_Araucanía,gx_Eólicas_Atacama,gx_Eólicas_Biobío,gx_Eólicas_Coquimbo,gx_Eólicas_Los Lagos,gx_Eólicas_O’Higgins,...,emb_POLCURA,emb_RALCO,emb_RAPEL,year,month,day,day_of_week,hour,date,rank
45746,2022-12-01 00:00:00,138.05,9186.975987,32.170000,502.420000,333.460000,311.270000,351.240000,45.360000,0.000000,...,731.78,722.97,104.19,2022,12,1,3,0,2022-12-01,15
45747,2022-12-01 01:00:00,149.41,8893.363555,30.540000,447.900000,346.070000,282.850000,305.310000,40.910000,0.000000,...,731.88,722.94,104.20,2022,12,1,3,1,2022-12-01,11
45748,2022-12-01 02:00:00,148.94,8652.044802,40.490000,389.670000,421.300000,251.490000,282.060000,94.030000,0.710000,...,731.95,722.93,104.20,2022,12,1,3,2,2022-12-01,12
45749,2022-12-01 03:00:00,138.05,8548.414757,58.790000,475.870000,411.620000,275.380000,201.390000,109.110000,0.090000,...,732.02,722.92,104.21,2022,12,1,3,3,2022-12-01,16
45750,2022-12-01 04:00:00,140.48,8589.178114,95.590000,513.980000,337.600000,290.760000,164.960000,50.430000,0.000000,...,731.91,722.91,104.22,2022,12,1,3,4,2022-12-01,14
45751,2022-12-01 05:00:00,148.87,8626.619147,121.670000,539.550000,261.720000,293.870000,151.350000,24.750000,0.000000,...,731.84,722.88,104.23,2022,12,1,3,5,2022-12-01,13
45752,2022-12-01 06:00:00,172.57,8770.626843,158.820000,524.750000,157.130000,316.730000,58.880000,19.110000,0.000000,...,731.75,722.86,104.23,2022,12,1,3,6,2022-12-01,6
45753,2022-12-01 07:00:00,112.60,9017.223632,201.360000,526.300000,91.410000,284.820000,64.110000,12.290000,0.000000,...,732.03,722.86,104.24,2022,12,1,3,7,2022-12-01,20
45754,2022-12-01 00:00:00,138.05,9186.975987,32.170000,502.420000,333.460000,311.270000,351.240000,45.360000,0.000000,...,731.78,722.97,104.19,2022,12,1,3,0,2022-12-01,16
45755,2022-12-01 01:00:00,149.41,8893.363555,30.540000,447.900000,346.070000,282.850000,305.310000,40.910000,0.000000,...,731.88,722.94,104.20,2022,12,1,3,1,2022-12-01,8


In [34]:
augmented_df.to_csv(r'../data/df_final_metadata_classes.csv', index=False)